In [43]:
import pandas as pd
from pathlib import Path

# File mappings
file_model_pairs = {
    '../results/results_chatgpt.csv': 'GPTResponse',
    '../results/results_claude.csv': 'ClaudeResponse',
    '../results/results_llama31.csv': 'LLama31Response',
    #'../results/results_llama32.csv': 'LLama32Response',
    ##'../results/results_qwen.csv': 'QwenResponse',
    #'../results/results_phi.csv': 'PhiResponse',
    #'../results/results_smol.csv': 'SmollResponse',
    '../results/results_rf.csv': 'RFResponse',
    '../results/results_svm.csv': 'SVMResponse',
}

# Load first file
first_file = list(file_model_pairs.items())[0]
combined = pd.read_csv(first_file[0], sep=None, engine='python')
model_name = Path(first_file[0]).stem.split('_')[1]
combined = combined[['Prompt', first_file[1]]].rename(columns={first_file[1]: f"{model_name}_response"})

# Add other files
for file, column in list(file_model_pairs.items())[1:]:
    df = pd.read_csv(file, sep=None, engine='python')
    model_name = Path(file).stem.split('_')[1]
    df = df.rename(columns={column: f"{model_name}_response"})
    combined = pd.merge(combined, df[['Prompt', f"{model_name}_response"]], on='Prompt', how='left')

# Add metadata
meta_df = pd.read_csv('../results/combined_model_responses_meta.ssv', sep=None, engine='python')
combined = pd.merge(combined, meta_df[['Prompt', 'Metadata']], on='Prompt', how='left')

# Save results 
output_path = Path('../results/final/final-dataset.ssv')
output_path.parent.mkdir(parents=True, exist_ok=True)
combined.to_csv(output_path, index=False, sep=";")